## α-synucleïn test

We attempt to compare the results of my HPS implementation to Ramon's. First, lib imports : 

In [ ]:
import os
import sys
import importlib
if '../utils' not in sys.path:
    sys.path.insert(0,'../utils')
import lmp
import lmpsetup
import analysis

importlib.reload(lmp);
importlib.reload(lmpsetup);
importlib.reload(analysis);

seq = 'MDVFMKGISKAKEGVVAAAEKTKQGVAEAAGKTKEGVIYVGSKTKEGVVHGVATVAEKTKEQVTNVGGAVVTGVTAVAQKTVEGAGSLAAATGFVKKDQIGKNEEGAPQEGIIEDMPVDPDNEAYEMPSEEGYQDYEPEA'
pdb = '/home/adria/perdiux/prod/lammps/dignon/TEST/asyn/TEST/ramon_run/data_trj.pdb'

Considering only a single, simple and short LAMMPS run, **Ramon's** data gives :

In [2]:
ramon_dir = '/home/adria/perdiux/prod/lammps/dignon/TEST/asyn/TEST/ramon_run'
r_lmp = lmp.LMP(ramon_dir)
r_lmp.run('lmp.lmp', n_cores=1);
data = r_lmp.get_lmp_data()
data

array([[[   0.      ,  313.78388 ,  124.29978 ,  300.      ],
        [ 500.      ,   14.373451,  123.10149 ,  297.10792 ],
        [1000.      ,   16.663285,  118.2894  ,  285.49382 ],
        [1500.      ,   22.543052,  121.06175 ,  292.18495 ],
        [2000.      ,   26.739724,  130.07105 ,  313.92907 ]]])

Compare this to **my** HPS implementation, using the same starting frame and the same random seeds:

In [3]:
my_dir = '/home/adria/perdiux/prod/lammps/dignon/TEST/asyn/TEST/my_run_compare'
my_setup = lmpsetup.LMPSetup(oliba_wd=my_dir, seq=seq)
my_setup.t = 2000
my_setup.save = 500
my_setup.get_pdb_xyz(pdb)
my_setup.get_hps_params()
my_setup.get_hps_pairs()
my_setup.write_hps_files()

my_setup.run('lmp.lmp', n_cores=1);
my_data = my_setup.get_lmp_data()
my_data

array([[[   0.      ,  313.78387 ,  124.29978 ,  300.      ],
        [ 500.      ,   14.373443,  123.10149 ,  297.10792 ],
        [1000.      ,   16.663281,  118.28939 ,  285.49382 ],
        [1500.      ,   22.543012,  121.06174 ,  292.18492 ],
        [2000.      ,   26.739396,  130.07117 ,  313.92938 ]]])

Use LAMMPS ordering in my run we see that **nothing** actually changes

In [4]:
lmp_my_dir = '/home/adria/perdiux/prod/lammps/dignon/TEST/asyn/TEST/my_run_lmp'
lmp_my_dir_setup = lmpsetup.LMPSetup(oliba_wd=lmp_my_dir, seq=seq)
lmp_my_dir_setup.t = 2000
lmp_my_dir_setup.save = 500
lmp_my_dir_setup.del_missing_aas()
lmp_my_dir_setup.lammps_ordering()
lmp_my_dir_setup.get_pdb_xyz(pdb)
lmp_my_dir_setup.get_hps_params()
lmp_my_dir_setup.get_hps_pairs()
lmp_my_dir_setup.write_hps_files()

lmp_my_dir_setup.run('lmp.lmp', n_cores=1);
lmp_my_data = lmp_my_dir_setup.get_lmp_data()
lmp_my_data

array([[[   0.      ,  313.78387 ,  124.29978 ,  300.      ],
        [ 500.      ,   14.373443,  123.10149 ,  297.10792 ],
        [1000.      ,   16.663281,  118.28939 ,  285.49382 ],
        [1500.      ,   22.543012,  121.06174 ,  292.18492 ],
        [2000.      ,   26.739396,  130.07117 ,  313.92938 ]]])

So we get the same results, therefore the differences are somewhere else. **They are actually in a lambda_ij roundup difference between both run's**:

In [5]:
l_my_dir = '/home/adria/perdiux/prod/lammps/dignon/TEST/asyn/TEST/my_run_lambda'
l_lmp_my_dir_setup = lmpsetup.LMPSetup(oliba_wd=l_my_dir, seq=seq)
l_lmp_my_dir_setup.t = 2000
l_lmp_my_dir_setup.save = 500
l_lmp_my_dir_setup.del_missing_aas()
l_lmp_my_dir_setup.lammps_ordering()
l_lmp_my_dir_setup.get_pdb_xyz(pdb)
l_lmp_my_dir_setup.get_hps_params()
l_lmp_my_dir_setup.get_hps_pairs(from_file='/home/adria/scripts/depured-lammps/data/lambda_pairs.dat')
l_lmp_my_dir_setup.write_hps_files()

l_lmp_my_dir_setup.run('lmp.lmp', n_cores=1);
l_lmp_my_data = l_lmp_my_dir_setup.get_lmp_data()
l_lmp_my_data

array([[[   0.      ,  313.78388 ,  124.29978 ,  300.      ],
        [ 500.      ,   14.373451,  123.10149 ,  297.10792 ],
        [1000.      ,   16.663285,  118.2894  ,  285.49382 ],
        [1500.      ,   22.543052,  121.06175 ,  292.18495 ],
        [2000.      ,   26.739724,  130.07105 ,  313.92907 ]]])

In [6]:
l_lmp_my_data == data

array([[[ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True]]])

In [7]:
l_lmp_my_data == lmp_my_data

array([[[ True, False,  True,  True],
        [ True, False,  True,  True],
        [ True, False, False,  True],
        [ True, False, False, False],
        [ True, False, False, False]]])

In [8]:
l_lmp_my_data == my_data

array([[[ True, False,  True,  True],
        [ True, False,  True,  True],
        [ True, False, False,  True],
        [ True, False, False, False],
        [ True, False, False, False]]])